In [1]:
import json

# import the smartpynector module
from NBs import smartpynector as sp

In [2]:
# Defining constants
# Knowledge engine REST API URL
KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
SUBSCRIBER_ID = "http://127.0.0.1:8000/subscriber"

In [3]:
# Defining the smart connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=SUBSCRIBER_ID,
    knowledgeBaseName="Subscriber2",
    knowledgeBaseDescription="Subscriber2",
    reasonerEnabled=False,
)

In [4]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

2023-05-02 15:32:13 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-02 15:32:14 DEBUG http://127.0.0.1:8280 "POST /rest/sc HTTP/1.1" 200 0
2023-05-02 15:32:14 INFO Request successful
2023-05-02 15:32:14 DEBUG 


<Response [200]>

In [5]:
graph_pattern='''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .'''

In [6]:
ask_ki = sp.register_knowledge_interaction(
    knowledge_interaction_type="AskKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    },
    graph_pattern=graph_pattern,
    headers={
        "Content-Type": "application/json",
        "knowledge-Base-Id": SUBSCRIBER_ID,
    },
)
ask_ki

2023-05-02 15:32:14 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-02 15:32:15 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ki HTTP/1.1" 200 110
2023-05-02 15:32:15 INFO Request successful
2023-05-02 15:32:15 DEBUG {"knowledgeInteractionId":"http://127.0.0.1:8000/subscriber/interaction/54939002-1353-4ac0-82ba-f0f4cb51eff9"}


{'knowledgeInteractionId': 'http://127.0.0.1:8000/subscriber/interaction/54939002-1353-4ac0-82ba-f0f4cb51eff9',
 'prefixes': {'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
  'saref': 'https://w3id.org/saref#'}}

In [7]:
ask_ki_id = ask_ki["knowledgeInteractionId"]
ask_ki_id

'http://127.0.0.1:8000/subscriber/interaction/54939002-1353-4ac0-82ba-f0f4cb51eff9'

In [8]:
ask_ki_prefixes = ask_ki["prefixes"]
ask_ki_prefixes

{'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
 'saref': 'https://w3id.org/saref#'}

In [9]:
binding_set = sp.perform_ask_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=SUBSCRIBER_ID,
    knowledge_interaction_id=ask_ki_id,
    ask_data=[]
).json()["bindingSet"]

2023-05-02 15:32:15 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-02 15:32:15 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ask HTTP/1.1" 200 829
2023-05-02 15:32:15 INFO Request successful
2023-05-02 15:32:15 DEBUG {"bindingSet":[{"room_id":"\"http://127.0.0.1:8000/thermostat/rooms/1\"","temp":"23.5","device_id":"\"http://127.0.0.1:8000/thermostat/devices/1\"","meas":"<http://127.0.0.1:8000/thermostat/measurements/1>","timestamp":"\"2023-05-02T15:32:12+02:00\""}],"exchangeInfo":[{"initiator":"knowledgeBase","knowledgeBaseId":"http://127.0.0.1:8000/thermostat","knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/16e3f0a5-2d11-4339-8b34-a22b1bdcdf90","exchangeStart":"2023-05-02T13:32:15.642+00:00","exchangeEnd":"2023-05-02T13:32:15.784+00:00","status":"SUCCEEDED","bindingSet":[{"room_id":"\"http://127.0.0.1:8000/thermostat/rooms/1\"","temp":"23.5","device_id":"\"http://127.0.0.1:8000/thermostat/devices/1\"","meas":"<http://127.0.0.1:8000/thermostat/measur

In [10]:
binding_set

[{'room_id': '"http://127.0.0.1:8000/thermostat/rooms/1"',
  'temp': '23.5',
  'device_id': '"http://127.0.0.1:8000/thermostat/devices/1"',
  'meas': '<http://127.0.0.1:8000/thermostat/measurements/1>',
  'timestamp': '"2023-05-02T15:32:12+02:00"'}]

In [11]:
readURL = "https://graphdb.odissei.nl/repositories/MateuszTest"
writeURL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

sp.store_data_in_graphdb(
    graph_pattern=graph_pattern,
    binding_set=binding_set,
    prefixes=ask_ki_prefixes,
    read_url=readURL,
    write_url=writeURL,
)

<http://127.0.0.1:8000/thermostat/measurements/1> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://w3id.org/saref#Measurement> .
<http://127.0.0.1:8000/thermostat/measurements/1> <https://w3id.org/saref#hasValue> "23.5" .
<http://127.0.0.1:8000/thermostat/measurements/1> <https://w3id.org/saref#isMeasuredIn> <https://w3id.org/saref#TemperatureUnit> .
<http://127.0.0.1:8000/thermostat/measurements/1> <https://w3id.org/saref#hasTimestamp> "2023-05-02T15:32:12+02:00" .
<http://127.0.0.1:8000/thermostat/measurements/1> <https://w3id.org/saref#isMeasurementOf> <http://127.0.0.1:8000/thermostat/rooms/1> .
<http://127.0.0.1:8000/thermostat/measurements/1> <https://w3id.org/saref#relatesToProperty> <https://w3id.org/saref#Temperature> .
<http://127.0.0.1:8000/thermostat/measurements/1> <https://w3id.org/saref#measurementMadeBy> <http://127.0.0.1:8000/thermostat/devices/1> .
@prefix ns1: <https://w3id.org/saref#> .

<http://127.0.0.1:8000/thermostat/measurements/1> a ns1:Measurement ;